In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.callbacks import get_openai_callback

set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)

/var/folders/q5/m8wmkz_d1c11pbrck3p2lm_m0000gn/T/ipykernel_57615/1724076709.py:79: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  a = chat.predict("What is the recipe for soju")


Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of sugar

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and steam it until it is fully cooked.
3. Let the rice cool down to room temperature.
4. In a large bowl, mix the nuruk and sugar with the cooked rice.
5. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.
6. After the fermentation process is complete, strain the mixture through a cheesecloth to remove any solids.
7. Transfer the liquid to a clean container and let it sit for another 1-2 days to allow the flavors to develop.
8. Your homemade soju is now ready to be enjoyed! Serve chilled or at room temperature. 

Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 1 tablespoon sugar
- 1 teaspoon salt
- 2 tablespoons olive oil

Instructions:
1. In a small bowl, combine the warm water, sugar, a